# Working with proxies in EF Core
Link to the article: https://learn.microsoft.com/en-us/ef/ef6/fundamentals/proxies

## What is a proxy? 
When creating instances of POCO entity types, Entity Framework often creates instances of a dynamically generated derived type that acts as a proxy for the entity. This proxy overrides some virtual properties of the entity to insert hooks for performing actions automatically when the property is accessed.

In [23]:
#r "nuget: Microsoft.EntityFrameworkCore, 7.0.0"
#r "nuget: Microsoft.EntityFrameworkCore.Proxies, 7.0.0"
#r "nuget: Microsoft.EntityFrameworkCore.InMemory, 7.0.0"

Installed Packages Microsoft.EntityFrameworkCore, 7.0.0 Microsoft.EntityFrameworkCore.InMemory, 7.0.0 Microsoft.EntityFrameworkCore.Proxies, 7.0.0

In [24]:
using Microsoft.EntityFrameworkCore;
using Microsoft.Extensions.Logging;
using System;
using System.Reflection;

public class Post {
    public int Id {get; set;}
}

In [27]:
// Db context without proxy
public class DefaultContext : DbContext
{
    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder) {
        optionsBuilder
            .UseInMemoryDatabase("default");
    }

    protected override void OnModelCreating(ModelBuilder modelBuilder) {
        base.OnModelCreating(modelBuilder);

        modelBuilder.Entity<Post>().HasData(
            new Post { Id = 1},
            new Post { Id = 2}
        );
    }
    public DbSet<Post> Posts { get; set; }
}

{
    using DefaultContext context = new DefaultContext();
    context.Database.EnsureDeleted();
    context.Database.EnsureCreated();
    
    var post = context.Posts.FirstOrDefault();

    Console.WriteLine(post.GetType().Name ?? "No post found");
}

Post


In [28]:
// Db context with lazy loading proxy
public class LazyLoadedContext : DbContext
{
    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder) {
        optionsBuilder
            .UseLazyLoadingProxies()
            .UseInMemoryDatabase("lazy");
    }

    protected override void OnModelCreating(ModelBuilder modelBuilder) {
        base.OnModelCreating(modelBuilder);

        modelBuilder.Entity<Post>().HasData(
            new Post { Id = 1},
            new Post { Id = 2}
        );
    }
    public DbSet<Post> Posts { get; set; }
}

{
    using LazyLoadedContext context = new LazyLoadedContext();
    context.Database.EnsureDeleted();
    context.Database.EnsureCreated();
    
    var post = context.Posts.FirstOrDefault();

    Console.WriteLine($"Proxy type: {post.GetType().Name}");
    Console.WriteLine($"Base type: {post.GetType().BaseType.Name}");
}

Proxy type: PostProxy_4
Base type: Post
